# Testing on Driving Data

In [ ]:
from dataset import PresampledDataset
from model import *

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
trajs = pd.read_pickle("data/trajectories_slim.pkl")

In [ ]:
# build traj dataset 
traj_mat = np.zeros((1106,33,8))
traj_mat_p = np.zeros((1106,33,8))
for i,label in enumerate(['x1', 'y1', 'xd1', 'yd1', 'x2', 'y2','xd2', 'yd2']):
    itr = -1
    for k,v in trajs[label].items():
        if k[1] == 0:
            itr += 1
        if k[1] < 33:
            traj_mat[itr,k[1],i] = v
        if (not (k[1] == 0)) and k[1]<34:
            traj_mat_p[itr,k[1]-1,i] = v

#shuffle this data
inds = np.random.choice(traj_mat.shape[0],traj_mat.shape[0])
traj_mat = traj_mat[inds,:,:]
traj_mat_p = traj_mat_p[inds,:,:]
            
N_train = 1000
N_test = 100
X = traj_mat[:1000,:,:]
Y = traj_mat_p[:1000,:,:] #- X

X_test = traj_mat[1000:1100,:,:]
Y_test = traj_mat_p[1000:1100,:,:] #- X_test

In [ ]:
dataset = PresampledDataset(X,Y)

## Create conditional model and train it on the data

In [ ]:
config = {'num_bijectors': 4,
          'bijector': 'RealNVP',
          'network_size': 64,
          'use_batchnorm': True,
          'y_dim': 8,
          'x_dim': 8,
          'nn_layers': [64,64,32],
          'activation': 'tanh',
          'sigma_eps': 1.0,
          'batch_size': 50,
          'data_horizon': 0,
          'test_horizon': 10,
          'learning_rate': 1e-3}

In [ ]:
g1 = tf.Graph()
sess1 = tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=g1)

In [ ]:
model = CondInvicuna(sess1, config)
model.construct_model()

In [ ]:
model.train(dataset, 5000)

Evaluate test likelihood of prior

In [ ]:
np.mean(np.log(model.get_likelihood(X_test[:50,0:0,:],Y_test[:50,0:0,:],X_test[:50,:,:],Y_test[:50,:,:])))